In [1]:
import jax
import jax.numpy as jnp
import numpy as np
import optax
from flax import linen as nn
from flax.training import train_state
from tqdm import tqdm
import pickle
import matplotlib.pyplot as plt

import jaxmetric as jm

In [2]:
key = jax.random.PRNGKey(0)

In [3]:
psi=0

poly = lambda x: x[0]**5 + x[1]**5 + x[2]**5 +x[3]**5 + x[4]**5 + psi* x[0]*x[1]*x[2]*x[3]*x[4]

In [4]:
points = np.array([[0.5457718 +0.42877685j, -0.96667378+0.00622714j, -0.2195527-0.23682649j, 0.01382411+0.10245113j, 1.+0.j]])        

In [6]:
met = jm.metrics.get_2form_FS_proj_prod((4,),[1],points)[0]
print(jnp.linalg.eigvals(met))
print(met - met.T.conj())

[ 3.9507091e-01-2.1800628e-09j -2.0519954e-08+6.9551569e-09j
  3.9507091e-01-2.9802322e-08j  3.9507088e-01-7.4505806e-09j
  3.9507097e-01+0.0000000e+00j]
[[0.+2.0996083e-09j 0.+0.0000000e+00j 0.+9.3132257e-10j 0.+0.0000000e+00j
  0.+0.0000000e+00j]
 [0.+0.0000000e+00j 0.-3.5405328e-11j 0.+3.7252903e-09j 0.+0.0000000e+00j
  0.+0.0000000e+00j]
 [0.+9.3132257e-10j 0.+3.7252903e-09j 0.-1.7049476e-10j 0.+0.0000000e+00j
  0.+0.0000000e+00j]
 [0.+0.0000000e+00j 0.+0.0000000e+00j 0.+0.0000000e+00j 0.+1.3237566e-11j
  0.+0.0000000e+00j]
 [0.+0.0000000e+00j 0.+0.0000000e+00j 0.+0.0000000e+00j 0.+0.0000000e+00j
  0.+0.0000000e+00j]]


In [9]:
met = jm.metrics.get_ref_metric((4,),[1],poly,points)[0]
print(jnp.linalg.eigvals(met))
print(jnp.abs(met - met.T.conj()))

[0.37495068+0.00258989j 0.41909572-0.00347542j 0.38945806+0.00532177j]
[[5.2134506e-03 3.0262821e-04 3.1780578e-06]
 [3.0262821e-04 6.9844737e-03 1.4895858e-07]
 [3.1780578e-06 1.4895858e-07 1.0643543e-02]]


In [13]:
abs(points[0])

array([0.69405795, 0.96669384, 0.32293989, 0.10337959, 1.        ])

In [14]:
jm.pullback.get_pullback(jnp.array([points[0]]),[4],poly)[0]

Array([[ 1.00000000e+00+0.0000000e+00j, -2.39019617e-01+1.1609333e-01j,
         0.00000000e+00+0.0000000e+00j,  0.00000000e+00+0.0000000e+00j,
         0.00000000e+00+0.0000000e+00j],
       [ 0.00000000e+00+0.0000000e+00j, -1.22598419e-02-2.1941222e-03j,
         1.00000000e+00+0.0000000e+00j,  0.00000000e+00+0.0000000e+00j,
         0.00000000e+00+0.0000000e+00j],
       [ 0.00000000e+00+0.0000000e+00j,  1.14102106e-04-6.3933039e-05j,
         0.00000000e+00+0.0000000e+00j,  1.00000000e+00+0.0000000e+00j,
         0.00000000e+00+0.0000000e+00j]], dtype=complex64)

In [15]:
jnp.outer(5 * points[0]**4, 1/(5 * points[0]**4))

Array([[ 1.0000000e+00-2.98023224e-08j, -2.3901963e-01+1.16093375e-01j,
         1.7248943e+01-1.25564127e+01j, -2.0281407e+03-1.18944580e+02j,
        -2.0605086e-01+1.06726699e-01j],
       [-3.3851607e+00-1.64419425e+00j,  1.0000000e+00+0.00000000e+00j,
        -7.9035637e+01+1.41448593e+01j,  6.6700146e+03+3.73730420e+03j,
         8.7299472e-01-2.24993732e-02j],
       [ 3.7893977e-02+2.75850166e-02j, -1.2259843e-02-2.19412125e-03j,
         1.0000000e+00-1.49011612e-08j, -7.3573235e+01-6.04535828e+01j,
        -1.0752144e-02-1.63961749e-03j],
       [-4.9137231e-04+2.88175797e-05j,  1.1410211e-04-6.39330465e-05j,
        -8.1138080e-03+6.66694622e-03j,  1.0000000e+00-2.98023224e-08j,
         9.8172088e-05-5.83804322e-05j],
       [-3.8265600e+00-1.98201597e+00j,  1.1447220e+00+2.95024998e-02j,
        -9.0891129e+01+1.38601828e+01j,  7.5250527e+03+4.47495654e+03j,
         1.0000000e+00+0.00000000e+00j]], dtype=complex64)